# Train ANN Model Alkaline ORR

In [ ]:
import random
import sklearn
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn import ensemble
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr, kendalltau
from sklearn.model_selection import LeaveOneOut
import pandas as pd
#%matplotlib 
###########import packages##########
import catboost
import xgboost
import lightgbm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import *
import pickle
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import  *
###########import packages##########
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
# from keras.wrappers import scikit_learn
from scikeras.wrappers import KerasClassifier, KerasRegressor
###########loading data##########
loo = LeaveOneOut()
import matplotlib.pyplot as plt

# ANN needs scalers to regularize input

In [ ]:
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
Minmaxsc  = MinMaxScaler(feature_range=(0, 1))
Minmaxsc2  = MinMaxScaler(feature_range=(0, 1))
Stdsc  = StandardScaler()
Stdsc2  = StandardScaler()
MAsc  = MaxAbsScaler()
MAsc2  = MaxAbsScaler()
Rsc  = RobustScaler()
Rsc2  = RobustScaler()

# Functions

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc

def compute_classification_metrics(y_true, y_pred, y_pred_proba, average_method='macro', multi_class_method='ovo'):

    if len(y_true.shape) > 1:  # Check if it's a multi-class problem
        auc = roc_auc_score(y_true, y_pred_proba, average=average_method, multi_class=multi_class_method)
    else:
        auc = roc_auc_score(y_true, y_pred_proba[:, 1])

    f1 = f1_score(y_true, y_pred, average=average_method)
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average=average_method)
    recall = recall_score(y_true, y_pred, average=average_method)
    print(f'AUC: {auc}, F1 Score: {f1}, Accuracy: {acc}, Precision: {precision}, Recall: {recall}')
    return auc,f1,acc,precision,recall
def plot_roc_curve(y_train, y_pred_train, y_test, y_pred_test, title):
    fig, ax = plt.subplots()  # This creates a new figure and axes

    for data, label, dataset in [(y_train, y_pred_train, 'Train'), (y_test, y_pred_test, 'Test')]:
        if len(data.shape) > 1:  # Check if it's a multi-class problem
            fpr = dict()
            tpr = dict()
            roc_auc = dict()
            for i in range(data.shape[1]):
                fpr[i], tpr[i], _ = roc_curve(data.iloc[:, i], label[:, i])
                roc_auc[i] = auc(fpr[i], tpr[i])
            
            for i in range(data.shape[1]):
                ax.plot(fpr[i], tpr[i], label=f'ROC curve of {dataset} class {i} (area = {roc_auc[i]:.2f})')
        else:
            # Assuming the second column of label array represents the positive class probabilities
            fpr, tpr, _ = roc_curve(data, label[:, 1])
            roc_auc = auc(fpr, tpr)
            ax.plot(fpr, tpr, label=f'{dataset} ROC curve (area = {roc_auc:.2f})')

    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f'ROC Curve - {title}')
    ax.legend(loc="lower right")
    return fig, ax  # Return the Figure and Axes


def gridsearch(model,param,algorithm_name,X,y):
    # For binary classification, you could use 'accuracy' or 'f1' as a scoring method
    # For multi-class classification, you might want to use 'accuracy' or 'f1_macro'
    grid = GridSearchCV(model,param_grid=param,scoring='accuracy',cv=5,n_jobs=-1,verbose=2)
    grid.fit(X,y)
    best_model=grid.best_estimator_
    best_score=grid.best_score_
    print('Best Classifier:',grid.best_params_,'Best Score:', best_score)
    return best_model,best_score
seed=283

In [ ]:
from sklearn.metrics import roc_curve

def predict_print_plot(data_input, data_output, algorithm_name, model, random_seed):
    # Prepare train and test data
    data_input_train, data_input_test, data_output_train, data_output_test = train_test_split(data_input, data_output, test_size=0.05, random_state=random_seed,stratify=data_output)

    # Predict labels for all data, train data, and test data
    y_pred_all = model.predict(data_input)
    y_pred_train = model.predict(data_input_train)
    y_pred_test = model.predict(data_input_test)

    # Predict probabilities for all data, train data, and test data
    y_pred_proba_all = model.predict_proba(data_input)
    y_pred_proba_train = model.predict_proba(data_input_train)
    y_pred_proba_test = model.predict_proba(data_input_test)
    
    # Print metrics
    print('Metrics for All Data:')
    all_perf=compute_classification_metrics(data_output, y_pred_all, y_pred_proba_all)
    print('Metrics for Test Data:')
    test_perf=compute_classification_metrics(data_output_test, y_pred_test, y_pred_proba_test)
    print('Metrics for Train Data:')
    train_perf=compute_classification_metrics(data_output_train, y_pred_train, y_pred_proba_train)

    # Plot ROC curve for both train and test data
    fig,ax=plot_roc_curve(data_output_train, y_pred_proba_train, data_output_test, y_pred_proba_test, title=algorithm_name)
    return fig,ax,test_perf

In [ ]:
def create_ANN_model_3layer(X, learning_rate, num_classes, regular_term=0.001, neuron_number=25, drop_out_rate=0.1):
    regularizer=keras.regularizers.l2(regular_term)
    model = Sequential() 
    model.add(Dense(neuron_number, input_dim=X.shape[1], kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))    
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    
    # The number of nodes in the last layer depends on the number of classes
    if num_classes == 2: # binary classification
        model.add(Dense(1, activation='sigmoid'))  
        model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate), metrics=['accuracy'])
    else: # multi-class classification
        model.add(Dense(num_classes, activation='softmax'))  
        model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate), metrics=['accuracy'])

    return model


In [ ]:
database=pd.read_csv('processed_database.csv')

In [ ]:
data_output_full=database.iloc[:,1]
data_input_full=database.iloc[:,2:]

In [ ]:
# Create data_output_full_l
data_output_full_l = data_output_full.map(lambda x: 0 if x < 0.83 else 1)
# Create data_output_full_m
data_output_full_m = data_output_full.map(lambda x: 0 if x < 0.88 else 1)
# Create data_output_full_h
data_output_full_h = data_output_full.map(lambda x: 0 if x < 0.93 else 1)
# Create data_output_full_multi
data_output_full_multi = np.where(data_output_full < 0.83, 0, np.where(data_output_full > 0.93, 2, 1))
data_output_full_multi_encoded = pd.get_dummies(data_output_full_multi, prefix='class')

In [ ]:
data_input_full_ANN=Stdsc.fit_transform(data_input_full)

In [ ]:
from sklearn.model_selection import KFold
def predict_and_print_cv(data_output,model,n_splits=5):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    results_all, results_test, results_train = [], [], []

    for train_index, test_index in kfold.split(data_input_full_ANN):
        X_train, X_test = data_input_full_ANN[train_index], data_input_full_ANN[test_index]
        if len(data_output.shape)>1:
            y_train, y_test = data_output.iloc[train_index], data_output.iloc[test_index]
        else:
            y_train, y_test = data_output[train_index], data_output[test_index]
        model.fit(X_train, y_train)  

        pred_train = model.predict(X_train)
        pred_test = model.predict(X_test)

        real_train = y_train
        real_test = y_test

        pred_all = model.predict(data_input_full_ANN)
        real_all = data_output
        
        y_pred_proba_all = model.predict_proba(data_input_full_ANN)
        y_pred_proba_train = model.predict_proba(X_train)
        y_pred_proba_test = model.predict_proba(X_test)
        
        
        results_all.append(compute_classification_metrics(real_all, pred_all,y_pred_proba_all))
        results_test.append(compute_classification_metrics(real_test, pred_test,y_pred_proba_test))
        results_train.append(compute_classification_metrics(real_train, pred_train,y_pred_proba_train))

    # Compute average metrics over all folds
    avg_results_all = np.mean(results_all, axis=0)
    avg_results_test = np.mean(results_test, axis=0)
    avg_results_train = np.mean(results_train, axis=0)

    print("Average metrics over all data: ", avg_results_all)
    print("Average metrics over test data: ", avg_results_test)
    print("Average metrics over train data: ", avg_results_train)

    return avg_results_all, avg_results_test, avg_results_train


In [ ]:
batch_size_list = [16,32,64]
optimizers_list=['adam','sgd']
epochs_list=[25,50,75]
param_ann = dict(batch_size=batch_size_list, 
                 epochs=epochs_list,
                optimizer=optimizers_list
                )


In [ ]:
os.environ['PYTHONHASHSEED']=str(1)
random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)
tf.compat.v1.set_random_seed(1)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
def train_model(data_output,task_type,seed):
    if len(data_output.shape)>1:
        num_classes = data_output.shape[1] # Number of columns in the encoded DataFrame
    else:
        num_classes = data_output.nunique() # Number of columns in the encoded DataFrame
    model_ANNClassifier = KerasClassifier(build_fn=create_ANN_model_3layer(X=data_input_full,num_classes=num_classes,learning_rate=0.001), verbose=0)
    X_train_ANN,X_test_ANN,y_train_ANN,y_test_ANN=train_test_split(data_input_full_ANN,data_output,test_size=0.05,random_state=seed,stratify=data_output)
    ANN_3layer,ANN_3layer_score=gridsearch(model_ANNClassifier,param_ann,'Artificial Neural Network_3layer',X_train_ANN,y_train_ANN)
    fig,ax,test_perf=predict_print_plot(data_input_full_ANN,data_output,task_type,ANN_3layer,seed)
    return ANN_3layer,fig,test_perf


In [ ]:
import os
from keras.models import load_model
for seed in range(1,100):
    if not os.path.exists(str(seed) + "_th_seed"):
        os.makedirs(str(seed) + "_th_seed")    
    print('low')
    ANN_model_l,fig_l,perf_l=train_model(data_output_full_l,'Lenient',seed)
    # Save the model and the figure
    ANN_model_l.model_.save(os.path.join(str(seed) + "_th_seed", 'ANN_model_l.h5'))
    fig_l.savefig(os.path.join(str(seed) + "_th_seed", 'fig_l.png'))
    
    print('middle')
    ANN_model_m,fig_m,perf_m=train_model(data_output_full_m,'Moderate',seed)
    # Save the model and the figure
    ANN_model_m.model_.save(os.path.join(str(seed) + "_th_seed", 'ANN_model_m.h5'))
    fig_m.savefig(os.path.join(str(seed) + "_th_seed", 'fig_m.png'))
    
    print('high')
    ANN_model_h,fig_h,perf_h=train_model(data_output_full_h,'Strict',seed)
    # Save the model and the figure
    ANN_model_h.model_.save(os.path.join(str(seed) + "_th_seed", 'ANN_model_h.h5'))
    fig_h.savefig(os.path.join(str(seed) + "_th_seed", 'fig_h.png'))
    
    print('multi')
    ANN_model_multi,fig_multi,perf_multi=train_model(data_output_full_multi_encoded,'Multi-Class',seed)
    # Save the model and the figure
    ANN_model_multi.model_.save(os.path.join(str(seed) + "_th_seed", 'ANN_model_multi.h5'))
    fig_multi.savefig(os.path.join(str(seed) + "_th_seed", 'fig_multi.png'))

    average_tuple = tuple(sum(t) / len(t) for t in zip(perf_l, perf_m, perf_h, perf_multi))

    print('seed is',seed,'average perf:',average_tuple)

In [ ]:
def create_ANN_model_3layer_cv_adam(X, learning_rate, num_classes, regular_term=0.001, neuron_number=25, drop_out_rate=0.1):
    regularizer=keras.regularizers.l2(regular_term)
    model = Sequential() 
    model.add(Dense(neuron_number, input_dim=X.shape[1], kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))    
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    
    # The number of nodes in the last layer depends on the number of classes
    if num_classes == 2: # binary classification
        model.add(Dense(1, activation='sigmoid'))  
        model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate), metrics=['accuracy'])
    else: # multi-class classification
        model.add(Dense(num_classes, activation='softmax'))  
        model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate), metrics=['accuracy'])

    return model
def create_ANN_model_3layer_cv_sgd(X, learning_rate, num_classes, regular_term=0.001, neuron_number=25, drop_out_rate=0.1):
    regularizer=keras.regularizers.l2(regular_term)
    model = Sequential() 
    model.add(Dense(neuron_number, input_dim=X.shape[1], kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))    
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    
    # The number of nodes in the last layer depends on the number of classes
    if num_classes == 2: # binary classification
        model.add(Dense(1, activation='sigmoid'))  
        model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(learning_rate), metrics=['accuracy'])
    else: # multi-class classification
        model.add(Dense(num_classes, activation='softmax'))  
        model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(learning_rate), metrics=['accuracy'])

    return model


In [ ]:
# Call the function for getting CV average metrics
num_classes=2
model_ANNClassifier_CV = KerasClassifier(build_fn=create_ANN_model_3layer_cv_adam(X=data_input_full,num_classes=num_classes,learning_rate=0.001), batch_size=64, epochs=25, verbose=0)
avg_results_all_l, avg_results_test_l, avg_results_train_l = predict_and_print_cv(data_output_full_l,model_ANNClassifier_CV)

In [ ]:
# Call the function for getting CV average metrics
num_classes=2
model_ANNClassifier_CV = KerasClassifier(build_fn=create_ANN_model_3layer_cv_adam(X=data_input_full,num_classes=num_classes,learning_rate=0.001),batch_size=64, epochs=75, verbose=0)
avg_results_all_m, avg_results_test_m, avg_results_train_m = predict_and_print_cv(data_output_full_m,model_ANNClassifier_CV)

In [ ]:
# Call the function for getting CV average metrics
num_classes=2
model_ANNClassifier_CV = KerasClassifier(build_fn=create_ANN_model_3layer_cv_adam(X=data_input_full,num_classes=num_classes,learning_rate=0.001),batch_size=64, epochs=25, verbose=0)
avg_results_all_h, avg_results_test_h, avg_results_train_h = predict_and_print_cv(data_output_full_h,model_ANNClassifier_CV)

In [ ]:
# Call the function for getting CV average metrics
num_classes=3
model_ANNClassifier_CV = KerasClassifier(build_fn=create_ANN_model_3layer_cv_adam(X=data_input_full,num_classes=num_classes,learning_rate=0.001),batch_size=64, epochs=25, verbose=0)
avg_results_all_multi, avg_results_test_multi, avg_results_train_multi = predict_and_print_cv(data_output_full_multi_encoded,model_ANNClassifier_CV)